<a href="https://colab.research.google.com/github/ayshaw/Dog-Breed-Project/blob/master/generating_xtrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%reset
from bs4 import BeautifulSoup
import requests
import re
import matplotlib.pylab as plt
import matplotlib.image as mping
import os
import pandas as pd
import numpy as np
import collections
import seaborn as sns
import multiprocessing
import cv2
from keras.utils import to_categorical
print(multiprocessing.cpu_count())
# Imports keras
import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Flatten, Dropout, UpSampling2D, GlobalAveragePooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam, SGD
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

from keras.applications import resnet50
!nvidia-smi
!ls "/content/drive/My Drive/"
os.chdir('/content/drive/My Drive/')
file_type=pd.read_csv('ds_all.csv',index_col=0)
file_type.head()


Once deleted, variables cannot be recovered. Proceed (y/[n])? y


Using TensorFlow backend.


2
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Wed Dec 12 01:50:13 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|=============================

,filename,type,annotations,breed,breed_class,ymin,ymax,xmin,xmax,train/test
0,n02085620-Chihuahua/n02085620_10074.jpg,1,Annotation/n02085620-Chihuahua/n02085620_10074,Chihuahua,Toy Group,10.0,498.0,25.0,276.0,test
1,n02085620-Chihuahua/n02085620_10131.jpg,1,Annotation/n02085620-Chihuahua/n02085620_10131,Chihuahua,Toy Group,9.0,493.0,49.0,393.0,test
2,n02085620-Chihuahua/n02085620_10621.jpg,1,Annotation/n02085620-Chihuahua/n02085620_10621,Chihuahua,Toy Group,43.0,250.0,142.0,335.0,train
3,n02085620-Chihuahua/n02085620_1073.jpg,1,Annotation/n02085620-Chihuahua/n02085620_1073,Chihuahua,Toy Group,27.0,498.0,0.0,312.0,test
4,n02085620-Chihuahua/n02085620_10976.jpg,1,Annotation/n02085620-Chihuahua/n02085620_10976,Chihuahua,Toy Group,104.0,452.0,90.0,242.0,train


In [0]:
# for start in np.array([10000,11000]):
#   end=start+1000
#   print(start)
#   x_train=np.empty((250,250,3,end-start),dtype='uint8')
#   for index,row in file_type.loc[file_type['train/test']=='train'].reset_index().iloc[start:end,:].iterrows():
#     print(index)
#     x_train[:,:,:,index-start]=cv2.imread('Images_scaled/'+row['filename'])
#   np.save("x_train_"+str(start)+'_'+str(end), x_train)
#   plt.imshow(x_train[:,:,:,0])

In [0]:
x_train_all = np.concatenate([np.load('x_train_0_1000.npy'),
                              np.load('x_train_1000_2000.npy'),
                              np.load('x_train_2000_3000.npy'),
                              np.load('x_train_3000_4000.npy'),
                              np.load('x_train_4000_5000.npy'),
                              np.load('x_train_5000_6000.npy'),
                              np.load('x_train_6000_7000.npy'),
                              np.load('x_train_7000_8000.npy'),
                              np.load('x_train_8000_9000.npy'),
                              np.load('x_train_9000_10000.npy'),
                              np.load('x_train_10000_11000.npy'),
                              np.load('x_train_11000_12000.npy')],
                              axis=3)
np.save('x_train_all',x_train_all)

In [0]:
# shuffle the x_train 
x_train_all=np.load('x_train_all.npy')
shuffle_index=np.random.randint(12000, size=12000)
x_train_shuffled=x_train_all[:,:,:,shuffle_index]
np.save('x_train_shuffled',x_train_shuffled)

# standardize the channels for x_train
x_train_shuffled = np.load('x_train_shuffled.npy')
x_train_scaled = x_train_shuffled.copy()
for i in range(3):
    x_train_scaled[:,:,:,i] = (x_train_shuffled[:,:,:,i]-np.mean(x_train_shuffled[:,:,:,i]))/np.std(x_train_shuffled[:,:,:,i])

In [0]:
# save out x_train final
np.save('x_train_scaled',x_train_scaled)